**Sommario**
- [FAQ - Domande frequenti emerse durante i corsi](#faq---domande-frequenti-emerse-durante-i-corsi)
  - [Qual è il significato delle icone di IntelliSense?](#qual-è-il-significato-delle-icone-di-intellisense?)
  - [Confronto di uguaglianza su valori in virgola mobile](#confronto-di-uguaglianza-su-valori-in-virgola-mobile)
    - [Uso di una soglia di tolleranza](#uso-di-una-soglia-di-tolleranza)
    - [Uso di tipi decimali](#uso-di-tipi-decimali)
    - [Uso della "rappresentazione intera"](#uso-della-rappresentazione-intera)
  - [*Memory overhead*: grandi serie numeriche in Python e come NumPy può essere d'aiuto](#*memory-overhead*-grandi-serie-numeriche-in-python-e-come-numpy-può-essere-d'aiuto)
  - [Operatore scelto dall'utente](#operatore-scelto-dall'utente)
  - [Verificare l'esistenza di una variabile](#verificare-l'esistenza-di-una-variabile)
  - [Come si può creare un binario *stand-alone* e ridistribuibile da uno script Python?](#come-si-può-creare-un-binario-*stand-alone*-e-ridistribuibile-da-uno-script-python?)
  - [`print()` flush parameter](#print-flush-parameter)

# FAQ - Domande frequenti emerse durante i corsi

## Qual è il significato delle icone di IntelliSense?

L'elenco lo trovi nella [documentazione ufficiale](https://code.visualstudio.com/docs/editor/intellisense#_types-of-completions).

## Confronto di uguaglianza su valori in virgola mobile

Abbiamo visto che con i numeri in virgola mobile, il valore memorizzato internamente dall'oggetto `float` potrebbe non essere esattamente quello che si pensa che sia (es. `0.1 + 0.2`). Per questo motivo, non è una buona pratica confrontare direttamente i valori in virgola mobile per ottenere l'uguaglianza esatta. Si consideri questo esempio:
```python
>>> x = 1.1 + 2.2
>>> x == 3.3
False
```
Le rappresentazioni interne (binarie) degli operandi di questa addizione non sono esattamente uguali a `1.1` e `2.2`, quindi non si può fare affidamento su `x` per confrontarlo esattamente con `3.3.`

### Uso di una soglia di tolleranza
Il modo migliore per determinare se due valori in virgola mobile sono "uguali" è calcolare se sono vicini l'uno all'altro, con una certa tolleranza. Guardate questo esempio:

```python
TOLLERANZA = 0.00001
x = 1.1 + 2.2
abs(x - 3.3) < TOLLERANZA  # -> True
```

Che in pratica è come scrivere:
```python
abs((1.1 + 2.2) - 3.3) < 0.00001  # -> True
```

`abs()` restituisce il valore assoluto. Se il valore assoluto della differenza tra i due numeri è inferiore alla tolleranza specificata, i due numeri sono abbastanza vicini da essere considerati uguali.

Può essere utili prepararsi una funzione ad hoc:

```python
TOLLERANZA = 0.00001

def quasi_uguali(a, b):
    return abs(a - b) < TOLLERANZA
```

### Uso di tipi decimali

Quando la precisione è critica e non si possono tollerare errori nei confronti, si può optare per l'uso dei tipi di dato `Decimal` o `Fraction`. Possono essere utili in applicazioni finanziarie, matematiche o scientifiche dove la precisione è fondamentale.

```python
from decimal import Decimal

a = Decimal('1.1')
b = Decimal('2.2')
c = Decimal('3.3')

a + b == c  # -> True
```


### Uso della "rappresentazione intera"

Per alcune applicazioni, è possibile convertire numeri in virgola mobile in interi, moltiplicandoli per un fattore di scala e arrotondandoli infine all'intero più vicino. Questo è un approccio pratico che funziona in molti ambiti quotidiani, quando non si hanno numeri molto grandi o molto piccoli ed è necessario un confronto esatto tra unità di misura discrete.

Seguendo l'esempio precedente, per mantenere una "tolleranza" di `0.00001`, potremmo moltiplicare per `100_000` e poi arrotondare all'intero tutti i valori su cui dobbiamo fare i confronti.

```python
SCALA = 100_000

a = 1.1
b = 2.2
r = 3.3

round((a + b) * SCALA) == round(r * SCALA)  # -> True
# 330000 == 330000
```

### Uso di `round`
La funzione `round(numero, ndigits)` arrotonda `numero` alla `ndigits` cifra decimale. L'uso di `round` può essere utile per approssimare il risultato a un certo numero di cifre decimali, il che può essere sufficiente per alcune applicazioni dove la precisione non è critica o dove i numeri arrotondati sono sufficientemente vicini agli originali per gli scopi del problema.

**Esempio:**
```python
round(0.1 + 0.2, 4) == 0.3  # True
```
Questo funziona bene in questo caso specifico, ma non garantisce che funzioni in tutti i casi, soprattutto con operazioni più complesse o con una catena più lunga di calcoli.

## *Memory overhead*: grandi serie numeriche in Python e come NumPy può essere d'aiuto

Supponiamo di voler memorizzare un elenco di numeri interi (ma lo stesso discorso vale per i float) in Python:

```python
lista_di_numeri = []
per i in range(1000000):
    lista_dei_numeri.append(i)
```

Questi numeri possono stare facilmente in un intero a 64 bit, quindi si potrebbe immaginare che Python memorizzi questi milioni di interi in non più di ~8MB: un milione di oggetti a 8 byte.

In realtà, Python utilizza circa 35 MB di RAM per memorizzare questi numeri. Perché? Perché i numeri interi di Python sono oggetti e gli oggetti hanno un notevole [overhead](https://it.wikipedia.org/wiki/Overhead) di memoria. Discorso simile per i float:

```
32 bit
int:   overhead = 10 bytes, value = 4 bytes
float: overhead =  8 bytes, value = 8 bytes

64 bit
int:   overhead = 20 bytes, value = 8 bytes
float: overhead = 16 bytes, value = 8 bytes
```

Se vogliamo bypassare questi problemi eliminando l'*overhead* (sovraccarico) e avere un controllo più a basso livello sui tipi di dato numerici ti consigli di utilizzare il modulo [NumPy](https://pypi.org/project/numpy/) o altri moduli simili studiati appositamente per la manipolazione di grandi serie numeriche.

## Operatore scelto dall'utente

In [ ]:
operatore_str = input()

if   operatore_str == '>':
    operatore = int.__gt__
elif operatore_str == '<':
    operatore = int.__lt__
elif operatore_str == '<=':
    operatore = int.__le__
elif operatore_str == '>=':
    operatore = int.__ge__
elif operatore_str == '==':
    operatore = int.__eq__
elif operatore_str == '!=':
    operatore = int.__ne__
else:
    raise TypeError('!!')
    
# operatore = int.__gt__ if condizione else int.__lt__

operatore(3, 5)

## Verificare l'esistenza di una variabile

In [1]:
myVar = object()

Per controllare l'esistenza di una variabile locale:

In [2]:
'myVar' in locals()

True

Per controllare l'esistenza di una variabile globale:

In [3]:
'myVar' in globals()

True

Per controllare se un oggetto ha un certo attrubuto o metodo:

In [5]:
hasattr(myVar, '__eq__')

True

Per ottenere una lista di tutti gli oggetti (attributi, metodi, funzioni, classi ecc.) contenuti in un namespace o in un oggetto:

In [6]:
dir(myVar)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

## Come si può creare un binario *stand-alone* e ridistribuibile da uno script Python?


Non è necessario compilare Python in codice C se si vuole solo rendere *stand alone* un programma, in modo che gli utenti possano scaricarlo ed eseguirlo senza dover prima installare la distribuzione Python (ed eventualmente anche tutte le nostre dipendenze). Esistono diversi strumenti che determinano l'insieme dei moduli richiesti da un programma e li legano a un binario Python per produrre un singolo eseguibile.

Uno di questi è lo strumento `freeze`, incluso nella cartella delle sorgenti di Python sotto "Tools/freeze". Converte il *bytecode* di Python in degli array C; con un compilatore C si possono incorporare tutti i moduli in un nuovo programma, che viene poi collegato ai moduli standard di Python.

Per approfondire questo argomento puoi [partire da qua](https://docs.python.org/3/faq/programming.html#how-can-i-create-a-stand-alone-binary-from-a-python-script
).

I seguenti pacchetti possono aiutare nella creazione di eseguibili per console e GUI:

- [Nuitka](https://nuitka.net/) (Cross-platform)
- [PyInstaller](https://pyinstaller.org/) (Cross-platform)
- [PyOxidizer](https://pyoxidizer.readthedocs.io/en/stable/) (Cross-platform)
- [cx_Freeze](https://marcelotduarte.github.io/cx_Freeze/) (Cross-platform)
- [py2app](https://github.com/ronaldoussoren/py2app) (macOS only)
- [py2exe](https://www.py2exe.org/) (Windows only)



## `print()` flush parameter

@see https://realpython.com/python-flush-print-output/

```python
# On unix xxecute as:
# $ python3 print_flush_countdown.py | cat
# On windows execute as:
# $ py print_flush_countdown.py | echo

from time import sleep

for second in range(3, 0, -1):
    print(second)
    sleep(1)
print("Go!")

for second in range(3, 0, -1):
    print(second, flush=True)
    sleep(1)
print("Go!")
```

Gli esempi seguenti sono da provare nella shell Python standard. Ipython sembra fare sempre flush.

In [ ]:
from time import sleep

for second in range(3, 0, -1):
    print(second)
    sleep(1)
print("Go!")

3
2
1


'Go!'

In [ ]:
for second in range(3, 0, -1):
    print(second, end=" ")
    sleep(1)
print("Go!")

3 2 1 

'Go!'

In [ ]:
for second in range(3, 0, -1):
    print(second, end=" ", flush=True)
    sleep(1)
print("Go!")

3 2 1 

'Go!'